In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re

In [3]:
#converting price from string to in
def convert_price(s):
    #Check if no price is listed
    if not s or "From" not in s:
        return np.nan
    #The prices are listed with "from" and a dollar sign, so to format as an int, these must be removed
    cleaned = s.replace("From", "").replace("$", "").strip()
    cleaned = cleaned.replace(",", "")
    try:
        return int(cleaned)
    except:
        return np.nan

In [4]:
#Generalized function to accept any team name and return info on all of events on vivid seats for that team
def scrape_vivid_performer(url, team_name):
    headers = {
        "User-Agent": None #Insert your user agent here
    }

    resp = requests.get(url, headers=headers)
    #Print the status of the request
    print(f"{team_name}: Status code", resp.status_code)

    soup = BeautifulSoup(resp.text, "html.parser")

    event_cards = soup.select('div[data-testid^="production-listing-"]')
    #Print the number of events found in the request results
    print(f"{team_name}: Found event cards:", len(event_cards))

    #Will hold the information from each event as a list of dictionaries
    rows = []

    for card in event_cards:
        #For each event, record the title, date, time, venue, and city by locating them in the text
        title_el = card.select_one(
            ".MuiTypography-root.MuiTypography-small-medium.styles_titleTruncate__XiZ53.mui-pc7loe"
        )
        date_el = card.select_one(
            ".MuiTypography-root.MuiTypography-small-bold.MuiTypography-noWrap.mui-1fmntk1"
        )
        time_el = card.select_one(
            ".MuiTypography-root.MuiTypography-caption.mui-1pgnteb"
        )
        venue_el = card.select_one(
            ".MuiTypography-root.MuiTypography-small-regular.styles_textTruncate__wsM3Q.mui-1insuh9"
        )
        city_el = card.select_one(
            ".MuiTypography-root.MuiTypography-small-regular.styles_textTruncate__wsM3Q.mui-1wl3fj7"
        )

        #The price is recorded, and then converted to an int to be stored separately
        price_el = card.find("span", string=lambda t: t and "From" in t)
        price_text = price_el.get_text(strip=True) if price_el else ""
        price_int = convert_price(price_text)

        #Find the event url
        link_el = card.find(
            "a",
            attrs={"data-testid": re.compile(r"production-listing-row-")}
        )

        #Record the event url
        if link_el and link_el.get("href"):
            href = link_el["href"]
            if href.startswith("/"):
                full_url = "https://www.vividseats.com" + href
            else:
                full_url = href
        else:
            full_url = url

        #Create a dictionary from the gathered data and add it on to everything already gathered
        rows.append({
            "platform":   "VividSeats",
            "team":       team_name,
            "event_title": title_el.get_text(strip=True) if title_el else "",
            "event_date":  date_el.get_text(strip=True) if date_el else "",
            "event_time":  time_el.get_text(strip=True) if time_el else "",
            "venue":       venue_el.get_text(strip=True) if venue_el else "",
            "city":        city_el.get_text(strip=True) if city_el else "",
            "price_text":  price_text,
            "price_int":   price_int,
            "url":         full_url,
        })

    return rows

In [5]:
#Go through months so all the results fit on one page
month_urls = {
    "https://www.vividseats.com/los-angeles-lakers-tickets--sports-nba-basketball/performer/483?startDate=2025-12-10&endDate=2026-01-31",
    "https://www.vividseats.com/los-angeles-lakers-tickets--sports-nba-basketball/performer/483?startDate=2026-02-01&endDate=2026-02-28",
    "https://www.vividseats.com/los-angeles-lakers-tickets--sports-nba-basketball/performer/483?startDate=2026-03-01&endDate=2026-03-31",
    "https://www.vividseats.com/los-angeles-lakers-tickets--sports-nba-basketball/performer/483?startDate=2026-04-01&endDate=2026-04-30"
}

all_rows = []

for url in month_urls:
    rows = scrape_vivid_performer(url, "Lakers")
    all_rows.extend(rows)

df = pd.DataFrame(all_rows)
print(df.head())
df.to_csv("vividseats_lakers_events.csv", index=False)
print("Saved vividseats_events.csv")

Lakers: Status code 200
Lakers: Found event cards: 6
Lakers: Status code 200
Lakers: Found event cards: 12
Lakers: Status code 200
Lakers: Found event cards: 23
Lakers: Status code 200
Lakers: Found event cards: 17
     platform    team                                  event_title event_date  \
0  VividSeats  Lakers  Los Angeles Lakers at Oklahoma City Thunder      Apr 2   
1  VividSeats  Lakers       Los Angeles Lakers at Dallas Mavericks      Apr 5   
2  VividSeats  Lakers  Oklahoma City Thunder at Los Angeles Lakers      Apr 7   
3  VividSeats  Lakers  Los Angeles Lakers at Golden State Warriors      Apr 9   
4  VividSeats  Lakers           Phoenix Suns at Los Angeles Lakers     Apr 10   

  event_time                          venue               city price_text  \
0     6:30pm                  Paycom Center  Oklahoma City, OK  From $123   
1     6:30pm  American Airlines Center - TX         Dallas, TX  From $167   
2     7:30pm               Crypto.com Arena    Los Angeles, CA   Fr